In [ ]:
from transformers import AutoTokenizer
from tokenization_llama_Lookup import LlamaTokenizer
import pandas as pd
import glob

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-hf')

tokenizer_MEDVOC = AutoTokenizer.from_pretrained('./Llama-2-7B-EBM-MEDVOC/EBM_Vocabs_MEDVOC_Llama2/25K_0.5_/')
tokenizer_MEDVOC_ABPE = LlamaTokenizer.from_pretrained('./Llama-2-7B-EBM-MEDVOC/EBM_Vocabs_MEDVOC_Llama2/25K_0.75_',
                                                             added_vocab = './Llama-2-7B-EBM-MEDVOC/EBM_Vocabs_MEDVOC_Llama2/25K_0.5_/added_vocab.txt',
                                                             use_fast=False)


tokenizer_filter = AutoTokenizer.from_pretrained('./Llama-2-7B-EBM-Filter/EBM_Filter_Vocabs_Llama2/20K')

tokenizer_filter_ABPE = LlamaTokenizer.from_pretrained('./Llama-2-7B-EBM-Filter/EBM_Filter_Vocabs_Llama2/20K',
                                                             added_vocab = './Llama-2-7B-EBM-Filter/EBM_Filter_Vocabs_Llama2/20K/added_vocab.txt',
                                                             use_fast=False)

tokenizer_lookup = LlamaTokenizer.from_pretrained('./Llama-2-7B-EBM-MedicalLookup-Fragment/EBM_Lookup_Vocabs_SplitMoreThan1_Also_Add_As_Subwords_Llama2/EBM_Lookup_300',
                                                 added_vocab = './Llama-2-7B-EBM-MedicalLookup-Fragment/EBM_Lookup_Vocabs_SplitMoreThan1_Also_Add_As_Subwords_Llama2/EBM_Lookup_300/added_vocab.txt',
                                                 use_fast=False)

In [ ]:
len(tokenizer), len(tokenizer_MEDVOC),len(tokenizer_MEDVOC_ABPE),len(tokenizer_filter), len(tokenizer_filter_ABPE), len(tokenizer_lookup)

In [ ]:
df_TGT  = pd.read_csv("../../../../../../Medical/PushToNeumann/CSV-Datasets/EBM-Test.csv")
list_rs = df_TGT['target_text'].to_list()
list_rs = [x+'||'+str(idx) for idx,x in enumerate(list_rs)]

gs_base = open('./EBM-Summaries/EBM_100_2_Llama2_Base.txt','r').read().split('\n++++++++++++++++++\n')
gs_llama3 = open('./EBM-Summaries/EBM_75_1_Llama3.1_Base.txt','r').read().split('\n++++++++++++++++++\n')

gs_cpt_without_vocab = open('./EBM-Summaries/EBM_75_2_Llama2_CPT_WithoutVocab.txt','r').read().split('\n++++++++++++++++++\n')

gs_cpt_medvoc = open('./EBM-Summaries/EBM_80_2__Llama2_MEDVOC_WithoutAdaptBPE.txt','r').read().split('\n++++++++++++++++++\n')
gs_cpt_medvoc_adaptbpe = open('./EBM-Summaries/EBM_80_2__Llama2_MEDVOC_WithAdaptBPE.txt','r').read().split('\n++++++++++++++++++\n')

gs_cpt_filtered_without_adaptbpe = open('./EBM-Summaries/EBM_80_2__Llama2_Filter_WithoutAdaptBPE.txt','r').read().split('\n++++++++++++++++++\n')
gs_cpt_filtered_adaptbpe = open('./EBM-Summaries/EBM_80_2__Llama2_Filter_WithAdaptBPE.txt','r').read().split('\n++++++++++++++++++\n')

gs_cpt_lookup = open('./EBM-Summaries/EBM_60_2_Llama2_Lookup300.txt','r').read().split('\n++++++++++++++++++\n')

In [ ]:
from collections import defaultdict
dict_cmp = defaultdict(list)

for gs,rs in zip(gs_base, list_rs):
    dict_cmp[rs].append(gs)

for gs,rs in zip(gs_llama3, list_rs):
    dict_cmp[rs].append(gs)

for gs,rs in zip(gs_cpt_without_vocab, list_rs):
    dict_cmp[rs].append(gs)

for gs,rs in zip(gs_cpt_medvoc, list_rs):
    dict_cmp[rs].append(gs)

for gs,rs in zip(gs_cpt_medvoc_adaptbpe, list_rs):
    dict_cmp[rs].append(gs)

for gs,rs in zip(gs_cpt_filtered_without_adaptbpe, list_rs):
    dict_cmp[rs].append(gs)

for gs,rs in zip(gs_cpt_filtered_adaptbpe, list_rs):
    dict_cmp[rs].append(gs)

for gs,rs in zip(gs_cpt_lookup, list_rs):
    dict_cmp[rs].append(gs)


In [ ]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
def RL(ref, hyp):
    scores = scorer.score(ref,hyp)
    return scores['rougeL'].fmeasure

In [ ]:
for rs,gss in dict_cmp.items():
  try:
    rl_base = RL(rs.split('||')[0], gss[0])
    rl_llama3 = RL(rs.split('||')[0], gss[1])
    rl_cpt_without_vocab = RL(rs.split('||')[0], gss[2])
    
    rl_cpt_medvoc = RL(rs.split('||')[0], gss[3])
    rl_cpt_medvoc_adaptbpe = RL(rs.split('||')[0], gss[4])
    
    rl_cpt_filtered_without_adaptbpe = RL(rs.split('||')[0], gss[5])
    rl_cpt_filtered_adaptbpe = RL(rs.split('||')[0], gss[6])
    
    rl_cpt_lookup = RL(rs.split('||')[0], gss[7])
    
    dict_cmp[rs].extend([rl_base,rl_llama3,rl_cpt_without_vocab,rl_cpt_medvoc,rl_cpt_medvoc_adaptbpe,rl_cpt_filtered_without_adaptbpe,rl_cpt_filtered_adaptbpe,rl_cpt_lookup])
    
  except: print(rs)



In [ ]:
list_src = df_TGT['input_text'].to_list()


In [ ]:
list_src[0], gs_base[0], list_rs[0]

In [ ]:
import re
pattern = r"^[A-Za-z▁]+$"

def OOV_frac(tokenizer,rs):
    tokens = rs.split()
    
    oov_frac = 0
    consider = 0
    oov_list = list()
    for token in tokens:
        if re.match(pattern,token): 
            consider += 1
            if len(tokenizer.tokenize(token)) > 1: 
                oov_frac += 1
                
    return oov_frac/consider

def Novel_frac(src,rs):
    novel_frac = 0
    novel_list = list()
    for tok in rs.split():
        if tok not in src: 
            novel_frac += 1
            novel_list.append(tok)
    
    return novel_frac/len(rs.split()), novel_list

In [ ]:
OOV_frac(tokenizer,list_rs[10]), list_rs[10]

In [ ]:
dict_src = defaultdict(list)
for rs,src in zip(list_rs,list_src): dict_src[rs].append(src)

list_del_key = list()
for key,val in dict_src.items():
    # print(key,val)
    if len(val[0]) == 0: 
        list_del_key.append(key)
        continue
        
    dict_src[key].extend(Novel_frac(val[0].lower(),key))
    dict_src[key].append(OOV_frac(tokenizer,val[0]))
    dict_src[key].append(OOV_frac(tokenizer_MEDVOC,val[0]))
    dict_src[key].append(OOV_frac(tokenizer_MEDVOC_ABPE,val[0]))
    dict_src[key].append(OOV_frac(tokenizer_filter,val[0]))
    dict_src[key].append(OOV_frac(tokenizer_filter_ABPE,val[0]))
    dict_src[key].append(OOV_frac(tokenizer_lookup,val[0]))

In [ ]:
list_rl_base = list()
list_rl_llama3 = list()
list_rl_cpt_without_vocab = list()
list_rl_cpt_medvoc = list()
list_rl_cpt_medvoc_adaptbpe = list()
list_rl_cpt_filtered_without_adaptbpe = list()
list_rl_cpt_filtered_adaptbpe = list()
list_rl_cpt_lookup = list()

list_oov_src_base = list()
list_oov_src_medvoc = list()
list_oov_src_medvoc_adaptbpe = list()
list_oov_src_filtered_without_adaptbpe = list()
list_oov_src_filtered_adaptbpe = list()
list_oov_src_lookup = list()

list_oov_rs_base = list()
list_oov_rs_medvoc = list()
list_oov_rs_medvoc_adaptbpe = list()
list_oov_rs_filtered_without_adaptbpe = list()
list_oov_rs_filtered_adaptbpe = list()
list_oov_rs_lookup = list()

list_len_rs_base = list()
list_len_rs_medvoc = list()
list_len_rs_medvoc_adaptbpe = list()
list_len_rs_filtered_without_adaptbpe = list()
list_len_rs_filtered_adaptbpe = list()
list_len_rs_lookup = list()

list_novel_rs = list()

list_gs_base = list()
list_gs_llama3 = list()
list_gs_cpt_without_vocab = list()
list_gs_cpt_medvoc = list()
list_gs_cpt_medvoc_adaptbpe = list()
list_gs_cpt_filtered_without_adaptbpe = list()
list_gs_cpt_filtered_adaptbpe = list()
list_gs_cpt_lookup = list()

list_rs = list()
list_sd = list()

for rs in dict_cmp:
  try:
    gen_base,gen_llama3,gen_cpt_without_vocab,gen_medvoc,gen_medvoc_adaptbpe,gen_cpt_filtered_without_adaptbpe,gen_cpt_filtered_adaptbpe,gen_cpt_lookup,rl_base,rl_llama3,rl_cpt_without_vocab,rl_cpt_medvoc,rl_cpt_medvoc_adaptbpe,rl_cpt_filtered_without_adaptbpe,rl_cpt_filtered_adaptbpe,rl_cpt_lookup = dict_cmp[rs]
    
    sd = dict_src[rs][0]
    
    len_rs_base = len(tokenizer.tokenize(rs))
    len_rs_medvoc = len(tokenizer_MEDVOC.tokenize(rs))
    len_rs_medvoc_adaptbpe = len(tokenizer_MEDVOC_ABPE.tokenize(rs))
    len_rs_filtered_without_adaptbpe = len(tokenizer_filter.tokenize(rs))
    len_rs_filtered_adaptbpe = len(tokenizer_filter_ABPE.tokenize(rs))
    len_rs_lookup = len(tokenizer_lookup.tokenize(rs))

    oov_sd_base = dict_src[rs][-6]
    oov_sd_medvoc = dict_src[rs][-5]
    oov_sd_medvoc_adaptbpe = dict_src[rs][-4]
    oov_sd_filtered_without_adaptbpe = dict_src[rs][-3]
    oov_sd_filtered_adaptbpe = dict_src[rs][-2]
    oov_sd_lookup= dict_src[rs][-1]

    oov_rs_base = OOV_frac(tokenizer,rs)
    oov_rs_medvoc = OOV_frac(tokenizer_MEDVOC,rs)
    oov_rs_medvoc_adaptbpe = OOV_frac(tokenizer_MEDVOC_ABPE,rs)
    oov_rs_filtered_without_adaptbpe = OOV_frac(tokenizer_filter,rs)
    oov_rs_filtered_adaptbpe = OOV_frac(tokenizer_filter_ABPE,rs)
    oov_rs_lookup = OOV_frac(tokenizer_lookup,rs)
    
    novel_rs = dict_src[rs][1]
    
    list_rl_base.append(rl_base)
    list_rl_llama3.append(rl_llama3)
    list_rl_cpt_without_vocab.append(rl_cpt_without_vocab)
    list_rl_cpt_medvoc.append(rl_cpt_medvoc)
    list_rl_cpt_medvoc_adaptbpe.append(rl_cpt_medvoc_adaptbpe)
    list_rl_cpt_filtered_without_adaptbpe.append(rl_cpt_filtered_without_adaptbpe)
    list_rl_cpt_filtered_adaptbpe.append(rl_cpt_filtered_adaptbpe)
    list_rl_cpt_lookup.append(rl_cpt_lookup)
    
    list_len_rs_base.append(len_rs_base)
    list_len_rs_medvoc.append(len_rs_medvoc)
    list_len_rs_medvoc_adaptbpe.append(len_rs_medvoc_adaptbpe)
    list_len_rs_filtered_without_adaptbpe.append(len_rs_filtered_without_adaptbpe)
    list_len_rs_filtered_adaptbpe.append(len_rs_filtered_adaptbpe)
    list_len_rs_lookup.append(len_rs_lookup)
    
    list_gs_base.append(gen_base)
    list_gs_llama3.append(gen_llama3)
    list_gs_cpt_medvoc.append(gen_medvoc)
    list_gs_cpt_medvoc_adaptbpe.append(gen_medvoc_adaptbpe)
    list_gs_cpt_without_vocab.append(gen_cpt_without_vocab)
    list_gs_cpt_filtered_without_adaptbpe.append(gen_cpt_filtered_without_adaptbpe)
    list_gs_cpt_filtered_adaptbpe.append(gen_cpt_filtered_adaptbpe)
    list_gs_cpt_lookup.append(gen_cpt_lookup)
    
    list_rs.append(rs)
    list_sd.append(sd)
    
    list_novel_rs.append(novel_rs)

    list_oov_rs_base.append(oov_rs_base)
    list_oov_rs_medvoc.append(list_oov_rs_medvoc)
    list_oov_rs_medvoc_adaptbpe.append(list_oov_rs_medvoc_adaptbpe)
    list_oov_rs_filtered_without_adaptbpe.append(oov_rs_filtered_without_adaptbpe)
    list_oov_rs_filtered_adaptbpe.append(oov_rs_filtered_adaptbpe)
    list_oov_rs_lookup.append(oov_rs_lookup)

    list_oov_src_base.append(oov_sd_base)
    list_oov_src_medvoc.append(oov_sd_medvoc)
    list_oov_src_medvoc_adaptbpe.append(oov_sd_medvoc_adaptbpe)
    list_oov_src_filtered_without_adaptbpe.append(oov_sd_filtered_without_adaptbpe)
    list_oov_src_filtered_adaptbpe.append(oov_sd_filtered_adaptbpe)
    list_oov_src_lookup.append(oov_sd_lookup)

  except Exception as e:
    print(e)
#     print(rs)
    print(len(dict_cmp[rs]), len(dict_src[rs]))
    pass

In [ ]:
import pandas as pd

df = pd.DataFrame({'SD': list_sd,'RS':list_rs, \
                   'GS_Base': list_gs_base , 'GS_Llama3.1': list_gs_llama3,'GS_CPT_Without_Vocab': list_gs_cpt_without_vocab, \
                   'GS_CPT_MedVoc': list_gs_cpt_medvoc, 'GS_CPT_MedVoc_AdaptBPE': list_gs_cpt_medvoc_adaptbpe, \
                   'GS_CPT_Filtering_WithoutAdaptBPE': list_gs_cpt_filtered_without_adaptbpe, \
                   'GS_CPT_Filtering_AdaptBPE': list_gs_cpt_filtered_adaptbpe, \
                   'GS_CPT_Lookup':list_gs_cpt_lookup, \
                   'RS_Len_Base': list_len_rs_base, 'RS_Len_Filtering_WithoutAdaptBPE': list_len_rs_filtered_without_adaptbpe, \
                   'RS_Len_MedVoc': list_len_rs_medvoc, 'RS_Len_MedVoc_AdaptBPE': list_len_rs_medvoc_adaptbpe, \
                   'RS_Len_Filtering_AdaptBPE': list_len_rs_filtered_adaptbpe, \
                   'RS_Len_Lookup' : list_len_rs_lookup, \
                   'OOV_RS_Base': list_oov_rs_base, 'OOV_RS_Filtering_WithoutAdaptBPE': list_oov_rs_filtered_without_adaptbpe, \
                   'OOV_RS_MedVoc': list_oov_rs_medvoc, 'OOV_RS_MedVoc_AdaptBPE': list_oov_rs_medvoc_adaptbpe, \
                   'OOV_RS_Filtering_AdaptBPE': list_oov_rs_filtered_adaptbpe, \
                   'OOV_RS_Lookup' : list_oov_rs_lookup, \
                   'Novel_RS': list_novel_rs, \
                   'OOV_SD_Base': list_oov_src_base, 'OOV_SD_Filtering_WithoutAdaptBPE': list_oov_src_filtered_without_adaptbpe, \
                   'OOV_SD_MedVoc': list_oov_src_medvoc, 'OOV_SD_MedVoc_AdaptBPE': list_oov_src_medvoc_adaptbpe, \
                   'OOV_SD_Filtering_AdaptBPE': list_oov_src_filtered_adaptbpe, \
                   'OOV_SD_Lookup' : list_oov_src_lookup, \
                   'R-L_Base': list_rl_base , 'R-L_Llama3.1': list_rl_llama3, 'R-L_CPT_Without_Vocab': list_rl_cpt_without_vocab, \
                   'R-L_CPT_MedVoc': list_rl_cpt_medvoc, 'R-L_CPT_MedVoc_AdaptBPE': list_rl_cpt_medvoc_adaptbpe, \
                   'R-L_CPT_Filtering_WithoutAdaptBPE': list_rl_cpt_filtered_without_adaptbpe, \
                   'R-L_CPT_Filtering_AdaptBPE': list_rl_cpt_filtered_adaptbpe, \
                   'R-L_CPT_Lookup':list_rl_cpt_lookup
                   })



In [ ]:
df.to_csv('./EBM-Summaries/EBM_Llama2_Compare_SelfAdaptBPE_WithLlama3.csv',index=False)

In [ ]:
import pandas as pd
df = pd.read_csv('./EBM-Summaries/EBM_Llama2_Compare_SelfAdaptBPE_WithLlama3.csv')

In [ ]:
import argparse
from quickumls import QuickUMLS
import pandas as pd
import glob


umls_path = '/Users/gunjanbalde/Documents/QuickUMLS_Files/'
matcher = QuickUMLS(umls_path,similarity_name='cosine',threshold=0.95) 

In [ ]:
from collections import defaultdict
import json
counter_TGT_RS = defaultdict(list)

lines_TGT = df['RS'].to_list()

for idx,abs in enumerate(lines_TGT):
    if idx%10 == 0: print(f'Processed till {idx+1}... ')

    flag = 0
    d = matcher.match(abs, best_match=True, ignore_syntax=False)
    if len(d) == 0:
        counter_TGT_RS[idx].append('happy')
    
    else:
        for l in d:
            counter_TGT_RS[idx].append(l[0]['ngram'])

import re
pattern = r"^[A-Za-z▁]+$"

counter_TGT_RS_Medical_Words = defaultdict(set)

for key,val in counter_TGT_RS.items():
    try:
      print('---------')
      for words in val:
        for word in words.split():
          print(word)
          if re.match(pattern,word.strip()): counter_TGT_RS_Medical_Words[key].add(word.strip())
    except: print("Error in", key,val)

for key,val in counter_TGT_RS_Medical_Words.items():
    print(key,val)
    print('-----------------')

import pickle as pkl
with open('./EBM-Summaries/EBM_Llama2_Compare_SelfAdaptBPE_MedicalWords.pkl','wb') as f:
    pkl.dump(counter_TGT_RS_Medical_Words,f)
f.close()


In [ ]:
len(counter_TGT_RS_Medical_Words)

In [ ]:
import pickle as pkl


with open('./EBM-Summaries/EBM_Llama2_Compare_SelfAdaptBPE_MedicalWords.pkl','rb') as f:
    counter_TGT_RS_Medical_Words = pkl.load(f)
f.close()

In [ ]:
list_medical_words_splitmorethan1 = list()
list_medical_words_splitmorethan2 = list()
list_medical_words_splitmorethan3 = list()

for idx, words in counter_TGT_RS_Medical_Words.items():
    total_words = len(words)
    splitmorethan1 = 0
    splitmorethan2 = 0
    splitmorethan3 = 0
    
    for word in words:
        if len(tokenizer.tokenize(word)) > 1: splitmorethan1 += 1
        if len(tokenizer.tokenize(word)) > 2: splitmorethan2 += 1
        if len(tokenizer.tokenize(word)) > 3: splitmorethan3 += 1
    
    list_medical_words_splitmorethan1.append(splitmorethan1/total_words)
    list_medical_words_splitmorethan2.append(splitmorethan2/total_words)
    list_medical_words_splitmorethan3.append(splitmorethan3/total_words)


In [ ]:
df['Medical_Words_SplitMoreThan1'] = list_medical_words_splitmorethan1
df['Medical_Words_SplitMoreThan2'] = list_medical_words_splitmorethan2
df['Medical_Words_SplitMoreThan3'] = list_medical_words_splitmorethan3

df.to_csv('./EBM-Summaries/EBM_Llama2_Compare_SelfAdaptBPE_MedicalWords_WithLlama3.csv',index=False)

In [5]:
import pandas as pd
df = pd.read_csv('./EBM-Summaries/EBM_Llama2_Compare_SelfAdaptBPE_MedicalWords_WithLlama3.csv')
df['Medical_Words_SplitMoreThan1'].describe(percentiles=[0.1,0.9])

count    424.000000
mean       0.408903
std        0.157894
min        0.000000
10%        0.223737
50%        0.400000
90%        0.600000
max        1.000000
Name: Medical_Words_SplitMoreThan1, dtype: float64

In [6]:
df_high_oov = df[df['Medical_Words_SplitMoreThan1'] >= 0.600000]
df_desc = pd.DataFrame()
for col in df_high_oov.columns:
      if col.startswith('CSr') or col.startswith('R-L') : df_desc[col] = df_high_oov[col]
df_desc.describe(percentiles=[0.5])

,R-L_Base,R-L_Llama3.1,R-L_CPT_Without_Vocab,R-L_CPT_MedVoc,R-L_CPT_MedVoc_AdaptBPE,R-L_CPT_Filtering_WithoutAdaptBPE,R-L_CPT_Filtering_AdaptBPE,R-L_CPT_Lookup,CSr_Base,CSr_Llama3.1,CSr_CPT_Without_Vocab,CSr_CPT_MedVoc,CSr_CPT_MedVoc_AdaptBPE,CSr_CPT_Filtering_WithoutAdaptBPE,CSr_CPT_Filtering_AdaptBPE,CSr_CPT_Lookup
count,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000
mean,0.162176,0.146550,0.166146,0.149923,0.149416,0.169029,0.158431,0.171178,0.171049,0.163421,0.205393,0.183331,0.183979,0.174193,0.179689,0.159918
std,0.094305,0.102310,0.094818,0.075169,0.071911,0.114788,0.075742,0.094957,0.161863,0.163681,0.176900,0.158942,0.154811,0.158202,0.170215,0.181406
min,0.030303,0.029851,0.031746,0.025641,0.045977,0.031250,0.047619,0.029412,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.156863,0.125000,0.140351,0.147368,0.143885,0.152672,0.149533,0.156250,0.129032,0.104651,0.187500,0.153846,0.181818,0.166667,0.148148,0.086957
max,0.558824,0.439024,0.412698,0.411765,0.405797,0.765432,0.400000,0.400000,0.700000,0.705882,0.666667,0.521739,0.562500,0.600000,0.562500,0.533333


## R-LCS
High Expert OOV Concentration (Terms Split more than once)
Base        Llama-3.1   W/O Vocab   MEDVOC      MEDVOC-ABPE Filter      Filter-ABPE Lookup
0.156863	0.125000    0.140351    0.147368	0.143885	0.152672	0.149533	0.156250

High Expert OOV Concentration (Terms Split more than thrice)
Base        Llama-3.1   W/O Vocab   MEDVOC      MEDVOC-ABPE Filter      Filter-ABPE Lookup
0.160000	0.147059    0.151899	0.146341	0.121951	0.145455	0.138889	0.153846

High Expert OOV Concentration (Terms Split more than once) --SD
Base        Llama-3.1   W/O Vocab   MEDVOC      MEDVOC-ABPE Filter      Filter-ABPE Lookup
0.149526	0.140423    0.146341	0.153052	0.155644	0.144454	0.149811	0.152681

High Expert OOV Concentration (Terms Split more than thrice) --SD
Base        Llama-3.1   W/O Vocab   MEDVOC      MEDVOC-ABPE Filter      Filter-ABPE Lookup
0.173333	0.147059    0.181818	0.181818	0.183673	0.172043	0.183908	0.166667

High-Novelty
Base        Llama-3.1   W/O Vocab   MEDVOC      MEDVOC-ABPE Filter      Filter-ABPE Lookup
0.112000	0.095238    0.117647	0.119048	0.105263	0.125000	0.123711	0.137931

Low-Novelty
Base        Llama-3.1   W/O Vocab   MEDVOC      MEDVOC-ABPE Filter      Filter-ABPE Lookup
0.223471    0.203448    0.243944	0.219244	0.217223	0.229206	0.235423	0.245091

----------------------------------------------------------------------------------
## CSr
High Expert OOV Concentration (Terms Split more than once)
Base        Llama-3.1   W/O Vocab   MEDVOC      MEDVOC-ABPE Filter      Filter-ABPE Lookup
0.129032	0.104651	0.187500	0.153846	0.181818	0.166667	0.148148	0.086957

High Expert OOV Concentration (Terms Split more than thrice)
Base        Llama-3.1   W/O Vocab   MEDVOC      MEDVOC-ABPE Filter      Filter-ABPE Lookup
0.117647	0.090909	0.129032	0.111111	0.133333	0.125000	0.095238	0.076923

High Expert OOV Concentration (Terms Split more than once) --SD
Base        Llama-3.1   W/O Vocab   MEDVOC      MEDVOC-ABPE Filter      Filter-ABPE Lookup
0.114379	0.109610	0.100251	0.125000	0.129167	0.129167	0.087121	0.111455

High Expert OOV Concentration (Terms Split more than thrice) --SD
Base        Llama-3.1   W/O Vocab   MEDVOC      MEDVOC-ABPE Filter      Filter-ABPE Lookup
0.111111	0.133333	0.142857	0.129032	0.142857	0.117647	0.125000	0.166667

High-Novelty
Base        Llama-3.1   W/O Vocab   MEDVOC      MEDVOC-ABPE Filter      Filter-ABPE Lookup
0.095238	0.090909	0.058824	0.090909	0.090909	0.100000	0.090909	0.086957

Low-Novelty
Base        Llama-3.1   W/O Vocab   MEDVOC      MEDVOC-ABPE Filter      Filter-ABPE Lookup
0.182488	0.219219	0.281746	0.326667	0.228758	0.272204	0.270270	0.246212

----------------------------------------------------------------------------------
Long RS (All)
Base        W/O Vocab   MEDVOC      MEDVOC-ABPE Filter      Filter-ABPE Lookup
0.182796	0.163934	0.180000	0.180905	0.175000	0.171779	0.157576

Low Expert OOV Concentration (Terms Split more than once)
Base        W/O Vocab   MEDVOC      MEDVOC-ABPE Filter      Filter-ABPE Lookup
0.158273	0.176471	0.189189	0.173913	0.195804	0.176991	0.162602

Low Expert OOV Concentration (Terms Split more than thrice)
Base        W/O Vocab   MEDVOC      MEDVOC-ABPE Filter      Filter-ABPE Lookup
0.175824	0.172840	0.165517	0.166667	0.171429	0.164948	0.178862

Low Expert OOV Concentration (Terms Split more than once) --SD
Base        W/O Vocab   MEDVOC      MEDVOC-ABPE Filter      Filter-ABPE Lookup
0.163636	0.170213	0.173516	0.167939	0.161905	0.183908	0.160920

Low Expert OOV Concentration (Terms Split more than thrice) --SD
Base        W/O Vocab   MEDVOC      MEDVOC-ABPE Filter      Filter-ABPE Lookup
0.173333	0.176000	0.180258	0.175676	0.185430	0.180328	0.177778


In [ ]:
df_long_rs = df[df['RS_Len_Base'] >= 212.700000]
df_long_rs.describe(percentiles=[0.5])

In [ ]:
df_RS_hHighOOV = df_high_oov[df_high_oov['RS_Len_Base']>=192]
df_RS_hHighOOV.describe(percentiles=[0.5])

In [ ]:
import pandas as pd
df = pd.read_csv('./EBM-Summaries/EBM_Llama2_Compare_SelfAdaptBPE_MedicalWords.csv')

In [ ]:
from collections import defaultdict

def getMatchInfo(list_matches):
    dict_concepts = defaultdict(list)
    for match_l in list_matches:
        for match_l_d in match_l:
            if match_l_d["preferred"] == 1: 
                cui = match_l_d['cui'] #the concept-id
                start = match_l_d['start'] 
                end = match_l_d['end']
                n_gram = match_l_d['ngram'].strip() #the surface form

                if '\n' in n_gram : continue
                key = str(start)+'_'+str(end)+'_'+n_gram

                if not cui in dict_concepts[key]: dict_concepts[key].append(cui)
                    
    str_ret = str()
    
    for key,val in dict_concepts.items():
        str_ret += str(key) + ':'
        for c_name in val:
            if not '\n' in c_name:
                str_ret += c_name + '|'
        str_ret += '\n'
    return (str_ret)


import numpy as np
import glob
import os
import pandas as pd

def check_overlap(concepts_ref, concepts_dec,verbose=False):
    list_concepts_ref = concepts_ref.splitlines()
    list_concepts_dec = concepts_dec.splitlines()
    
    cuis_dec, cuis_ref = list(), list()
    
    for line in list_concepts_ref:
        cui_ref = line.split(':')[1].split('|')[:-1]
        cuis_ref.extend(cui_ref)
    
    for line in list_concepts_dec:
        cui_dec = line.split(':')[1].split('|')[:-1]
        cuis_dec.extend(cui_dec)
    
    cuis_ref = set(cuis_ref)
    cuis_dec = set(cuis_dec)
    
    
    common_cuis = cuis_dec.intersection(cuis_ref)
    
    if verbose: print('Common_cuis:', common_cuis)
    
    if len(common_cuis) == 0: return 0.
    
    prec = len(common_cuis)/len(cuis_dec)
    rec = len(common_cuis)/len(cuis_ref)
    
    return ((2*prec*rec)/(prec+rec))
import numpy as np
import glob
import os
import pandas as pd

def check_overlap(concepts_ref, concepts_dec,verbose=False):
    list_concepts_ref = concepts_ref.splitlines()
    list_concepts_dec = concepts_dec.splitlines()
    
    cuis_dec, cuis_ref = list(), list()
    
    for line in list_concepts_ref:
        cui_ref = line.split(':')[1].split('|')[:-1]
        cuis_ref.extend(cui_ref)
    
    for line in list_concepts_dec:
        cui_dec = line.split(':')[1].split('|')[:-1]
        cuis_dec.extend(cui_dec)
    
    cuis_ref = set(cuis_ref)
    cuis_dec = set(cuis_dec)
    
    
    common_cuis = cuis_dec.intersection(cuis_ref)
    
    if verbose: print('Common_cuis:', common_cuis)
    
    if len(common_cuis) == 0: return 0.
    
    prec = len(common_cuis)/len(cuis_dec)
    rec = len(common_cuis)/len(cuis_ref)
    
    return ((2*prec*rec)/(prec+rec))
from numpy.random import seed
from numpy.random import rand
from numpy.random import randint
from numpy import mean
from numpy import median
from numpy import percentile
import numpy as np
def CIEval(f_score):
    dataset = np.array([x[1] for x in f_score])
    max_l = len(dataset)
    scores = list()
    for _ in range(1000):
        # bootstrap sample
        indices = randint(0, max_l, max_l)
        sample = dataset[indices]
        # calculate and store statistic
        statistic = mean(sample)
        scores.append(statistic)

    print('50th percentile (median) = %.4f' % median(scores))
    # calculate 95% confidence intervals (100 - alpha)
    alpha = 5.0
    # calculate lower percentile (e.g. 2.5)
    lower_p = alpha / 2.0
    # retrieve observation at lower percentile
    lower = max(0.0, percentile(scores, lower_p))
#     print('%.1fth percentile = %.4f' % (lower_p, lower))
    # calculate upper percentile (e.g. 97.5)
    upper_p = (100 - alpha) + (alpha / 2.0)
    # retrieve observation at upper percentile
    upper = min(1.0, percentile(scores, upper_p))
#     print('%.1fth percentile = %.4f' % (upper_p, upper))
    print('C.I. Window = %.4f '%max([upper-median(scores),median(scores)-lower]))
for column in ['GS_Base', 'GS_Llama3.1', 'GS_CPT_Without_Vocab', 'GS_CPT_MedVoc',
       'GS_CPT_MedVoc_AdaptBPE', 'GS_CPT_Filtering_WithoutAdaptBPE',
       'GS_CPT_Filtering_AdaptBPE', 'GS_CPT_Lookup']:

    print(f'--Processing {column}.....')

    f_score = list()
    for row_id in range(df.shape[0]):
        try:                
            dec_sum = df.iloc[row_id][column]
            ref_sum = df.iloc[row_id]['RS']
            ref_sum = ref_sum.split('||')[0]
            
            ref_con = getMatchInfo(matcher.match(ref_sum, best_match=True, ignore_syntax=False))
            dec_con = getMatchInfo(matcher.match(dec_sum, best_match=True, ignore_syntax=False))
            
            # print(f'REF_Sum: {ref_sum}')
            # print(f'DEC_Sum: {dec_sum}')

            # print(check_overlap(ref_con,dec_con))
            f_score.append((row_id,check_overlap(ref_con,dec_con)))
            
        except Exception as e: 
            print(e)
            
#         break
#     break
    df['CSr'+column[2:]] = [x[1] for x in f_score]
    CIEval(f_score)



In [ ]:
df.to_csv('./EBM-Summaries/EBM_Llama2_Compare_SelfAdaptBPE_MedicalWords_WithLlama3.csv',index=False)

In [ ]:
import pandas as pd
df = pd.read_csv('./EBM-Summaries/EBM_Llama2_Compare_SelfAdaptBPE_MedicalWords_WithLlama3.csv')
df['Medical_Words_SplitMoreThan1'].describe(percentiles=[0.9])

In [ ]:
df_high_oov = df[df['Medical_Words_SplitMoreThan1'] >=0.60]
df_desc = df_high_oov.drop(columns=['GS_Llama3.1' ,'GS_CPT_MedVoc',
       'GS_CPT_MedVoc_AdaptBPE', 'GS_CPT_Filtering_WithoutAdaptBPE',
       'GS_CPT_Filtering_AdaptBPE', 'RS_Len_Base',
       'RS_Len_Filtering_WithoutAdaptBPE', 'RS_Len_MedVoc',
       'RS_Len_MedVoc_AdaptBPE', 'RS_Len_Filtering_AdaptBPE', 'RS_Len_Lookup',
       'OOV_RS_Base', 'OOV_RS_Filtering_WithoutAdaptBPE', 'OOV_RS_MedVoc',
       'OOV_RS_MedVoc_AdaptBPE', 'OOV_RS_Filtering_AdaptBPE', 'OOV_RS_Lookup',
       'Novel_RS', 'OOV_SD_Base', 'OOV_SD_Filtering_WithoutAdaptBPE',
       'OOV_SD_MedVoc', 'OOV_SD_MedVoc_AdaptBPE', 'OOV_SD_Filtering_AdaptBPE',
       'OOV_SD_Lookup',
       'Medical_Words_SplitMoreThan1', 'Medical_Words_SplitMoreThan2',
       'Medical_Words_SplitMoreThan3', 'Medical_Words_SplitMoreThan1_SD',
       'Medical_Words_SplitMoreThan2_SD', 'Medical_Words_SplitMoreThan3_SD'])

df_desc = df_desc[df_desc['R-L_Base']<df_desc['R-L_CPT_Lookup']]
df_desc = df_desc[df_desc['R-L_CPT_Without_Vocab']<df_desc['R-L_CPT_Lookup']]
df_desc_greatest = df_desc.nlargest(int(20), 'R-L_CPT_Lookup')
for idx,row in df_desc_greatest.iterrows():
    print('------------------------'*8)
    print('+++SD',row['SD'])
    print('+++RS        :',row['RS'])
    print('######################################')
    print('**GS_Base    :',row['GS_Base'])
    print('**GS_W/oVocab:',row['GS_CPT_Without_Vocab'])
    print('**GS_Lookup  :',row['GS_CPT_Lookup'])

In [ ]:
from scipy.stats import wilcoxon
print(wilcoxon(df_high_oov['R-L_CPT_Without_Vocab'].to_list(),df_high_oov['R-L_CPT_Lookup'].to_list(),alternative='less'), \
wilcoxon(df_low_oov['R-L_CPT_Without_Vocab'].to_list(),df_low_oov['R-L_CPT_Lookup'].to_list(),alternative='less'))

In [ ]:
import re
pattern = r"^[A-Za-z▁]+$"

def OOV_frac(tokenizer,rs):
    tokens = rs.split()
    
    oov_frac = 0
    consider = 0
    oov_list = list()
    for token in tokens:
        if re.match(pattern,token): 
            consider += 1
            if len(tokenizer.tokenize(token)) > 1: 
                oov_frac += 1
                oov_list.append([token,tokenizer.tokenize(token)])
    return oov_frac/consider, oov_list

In [ ]:
list_common_words = []
df_high_oov_better = df_high_oov[df_high_oov['R-L_CPT_Filtering_WithoutAdaptBPE']>df_high_oov['R-L_CPT_Without_Vocab']]
print(df_high_oov_better.shape, df_high_oov.shape)
df_high_oov_better = df_high_oov_better.sort_values(by='R-L_CPT_Filtering_WithoutAdaptBPE',ascending=False)
for idx,row in df_high_oov_better.iterrows():
    rs = row['RS']
    print('++++++++++++', row['R-L_CPT_Filtering_WithoutAdaptBPE'], row['R-L_CPT_Without_Vocab'],row['Novel_RS'],row['OOV_RS_Base'],row['OOV_RS_Filtering_WithoutAdaptBPE'],row['RS_Len_Base'],row['RS_Len_Filtering_WithoutAdaptBPE'])
    print('-- SD: ',row['SD'])
    print('-- RS                               :',row['RS'])
    print('***')
    print('-- GS_CPT_Without_Vocab             :',row['GS_CPT_Without_Vocab'])
    print('-- GS_CPT_Without_Filtering_AdaptBPE:',row['GS_CPT_Filtering_WithoutAdaptBPE'])
    print('-- OOV Base Tokenizer               :', OOV_frac(tokenizer,rs)) 
    print('-- Lookup-1000                      :',OOV_frac(tokenizer_lookup,rs))
    common_words = 0
    for token1 in  row['GS_CPT_Filtering_WithoutAdaptBPE'].split():
        if token1 in row['GS_CPT_Without_Vocab']: common_words+=1
        
    common_words /= len(row['GS_CPT_Filtering_WithoutAdaptBPE'].split()) 
    
    list_common_words.append(common_words)
    print('-- Common Words Frac               :', common_words)
    

In [ ]:
df_high_oov_worst = df_high_oov[df_high_oov['R-L_CPT_Filtering_WithoutAdaptBPE']<=df_high_oov['R-L_CPT_Without_Vocab']]
list_common_words_worst = []
print(df_high_oov_worst.shape, df_high_oov.shape)
df_high_oov_worst = df_high_oov_worst.sort_values(by='R-L_CPT_Filtering_WithoutAdaptBPE')
for idx,row in df_high_oov_worst.iterrows():
    rs = row['RS']
    print('++++++++++++', row['R-L_CPT_Filtering_WithoutAdaptBPE'], row['R-L_CPT_Without_Vocab'],row['Novel_RS'],row['OOV_RS_Base'],row['OOV_RS_Filtering_WithoutAdaptBPE'],row['RS_Len_Base'],row['RS_Len_Filtering_WithoutAdaptBPE'])
    print('-- SD: ',row['SD'])
    print('-- RS                               : ',row['RS'])
    print('***')
    print('-- GS_CPT_Without_Vocab             :',row['GS_CPT_Without_Vocab'])
    print('-- GS_CPT_Without_Filtering_AdaptBPE:',row['GS_CPT_Filtering_WithoutAdaptBPE'])
    print('-- OOV Base Tokenizer               :', OOV_frac(tokenizer,rs)) 
    print('-- Filtering w/o AdaptBPE           :',OOV_frac(tokenizer_filtering,rs))
    common_words = 0
    for token1 in  row['GS_CPT_Filtering_WithoutAdaptBPE'].split():
        if token1 in row['GS_CPT_Without_Vocab']: common_words+=1
        
    common_words /= len(row['GS_CPT_Filtering_WithoutAdaptBPE'].split()) 
    
    list_common_words_worst.append(common_words)
    print('-- Common words Frac                :', common_words)

In [ ]:
df.describe(percentiles=[0.1,0.25,0.5,0.75,0.9])

In [ ]:
df_high_oov = df[df['OOV_RS_Lookup']>=0.27]
df_high_oov.describe()

df_long_rs = df[df['RS_Len_Lookup']>=93]
df_long_rs.describe()

In [ ]:
df_high_oov.describe()

In [ ]:
Long-RS:
Base > CPT_Filter_W/OABPE > CPT_W/O_Vocab > CPT_Lookup > CPT_Filter_AdaptBPE
0.219003 > 0.213338 > 0.208825 > 0.205657 > 0.185030

High-OOV:
CPT_Lookup > Base > CPT_Filter_AdaptBPE > CPT_Filter_W/OABPE > CPT_W/O_Vocab
0.263158 > 0.258094 > 0.255865 > 0.251984 > 0.245526

In [ ]:
df_high_oov = df[df['OOV_RS_Filtering_WithoutAdaptBPE']>=0.25]
df_high_oov.describe()

High-OOV:
Lookup25.59 > W/O_Vocab25.00 > Base24.83 > Filter_W/OABPE22.04 > Filter_ABPE20.97


In [ ]:
df_long_rs = df[df['RS_Len_Filtering_WithoutAdaptBPE']>=89]
df_long_rs.describe()

Long-RS:
Base20.64 > Lookup20.15 > W/O_Vocab19.33 > Filter_W/OABPE18.90 > Filter_ABPE18.27

In [ ]:
list_common_words = []
df_high_oov_better = df_high_oov[df_high_oov['R-L_CPT_Lookup']>df_high_oov['R-L_CPT_Without_Vocab']]
print(df_high_oov_better.shape, df_high_oov.shape)
df_high_oov_better = df_high_oov_better.sort_values(by='R-L_CPT_Lookup',ascending=False)
for idx,row in df_high_oov_better.iterrows():
    rs = row['RS']
    print('++++++++++++', row['R-L_CPT_Lookup'], row['R-L_CPT_Without_Vocab'],row['Novel_RS'],row['OOV_RS_Base'],row['OOV_RS_Lookup'],row['RS_Len_Base'],row['RS_Len_Lookup'])
    print('-- SD: ',row['SD'])
    print('-- RS                               :',row['RS'])
    print('***')
    print('-- GS_CPT_Without_Vocab             :',row['GS_CPT_Without_Vocab'])
    print('-- GS_CPT_Without_Filtering_AdaptBPE:',row['GS_CPT_Lookup'])
    print('-- OOV Base Tokenizer               :', OOV_frac(tokenizer,rs)) 
    print('-- Lookup-1000                      :',OOV_frac(tokenizer_lookup,rs))
    common_words = 0
    for token1 in  row['GS_CPT_Lookup'].split():
        if token1 in row['GS_CPT_Without_Vocab']: common_words+=1
        
    common_words /= len(row['GS_CPT_Lookup'].split()) 
    
    list_common_words.append(common_words)
    print('-- Common Words Frac               :', common_words)
    

In [ ]:
list_common_words = []
df_high_oov_worst = df_high_oov[df_high_oov['R-L_CPT_Lookup']<=df_high_oov['R-L_CPT_Without_Vocab']]
print(df_high_oov_worst.shape, df_high_oov.shape)
df_high_oov_better = df_high_oov_worst.sort_values(by='R-L_CPT_Lookup',ascending=False)
for idx,row in df_high_oov_worst.iterrows():
    rs = row['RS']
    print('++++++++++++', row['R-L_CPT_Lookup'], row['R-L_CPT_Without_Vocab'],row['Novel_RS'],row['OOV_RS_Base'],row['OOV_RS_Lookup'],row['RS_Len_Base'],row['RS_Len_Lookup'])
    print('-- SD: ',row['SD'])
    print('-- RS                               :',row['RS'])
    print('***')
    print('-- GS_CPT_Without_Vocab             :',row['GS_CPT_Without_Vocab'])
    print('-- GS_CPT_Without_Filtering_AdaptBPE:',row['GS_CPT_Lookup'])
    print('-- OOV Base Tokenizer               :', OOV_frac(tokenizer,rs)) 
    print('-- Lookup-1000                      :',OOV_frac(tokenizer_lookup,rs))
    common_words = 0
    for token1 in  row['GS_CPT_Lookup'].split():
        if token1 in row['GS_CPT_Without_Vocab']: common_words+=1
        
    common_words /= len(row['GS_CPT_Lookup'].split()) 
    
    list_common_words.append(common_words)
    print('-- Common Words Frac               :', common_words)
    